In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Colab-ready Python launcher for KNNSaver / run_clm_chat.py
This replaces the bash file and mounts Google Drive automatically.
"""

import os
import subprocess
from datetime import datetime

# ======================================
# 1. Mount Google Drive
# ======================================
print("[INFO] Mounting Google Drive...")
from google.colab import drive
drive.mount("/content/drive")


[INFO] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
!pip install transformers datasets accelerate sentencepiece faiss-cpu
!pip install einops

In [11]:
!pip install handystuff

In [ ]:
import os

# Re-define variables as they are in the user's code, or ensure they are accessible
# This assumes the user's cell '_RXz57WN7aFL' has been run or the variables are defined elsewhere.
# For safety, re-defining them here ensures this check cell works independently.

MODEL = "allenai/OLMo-2-0425-1B-Instruct"
BASE_DIR = "/content/drive/MyDrive/faiss_mts_olmo/"  #Replace with local google drive file path

SCRIPT = os.path.join(BASE_DIR + 'core_scripts/', "run_clm_chat.py")
TRAIN_FILE = os.path.join(BASE_DIR + 'execution_scripts/mock_datasets_radiology_jsonl/', "mimic_inspired_train_context_impression-finetune.jsonl")
VAL_FILE = os.path.join(BASE_DIR + 'execution_scripts/mock_datasets_radiology_jsonl/', "mimic_inspired_val_context_impression-finetune.jsonl")

#Replace with your google drive file path
OUT_DIR = os.path.join(BASE_DIR + 'execution_scripts/', f"outputs/{MODEL.replace('/','_')}")
DSTORE_DIR = os.path.join(BASE_DIR + 'execution_scripts/', f"datastores/{MODEL.replace('/','_')}")

LOG_DIR = os.path.join(BASE_DIR + 'execution_scripts/', "logs")

# Make directories
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(DSTORE_DIR, exist_ok=True)

print(f"Checking existence for:")
print(f"SCRIPT ({SCRIPT}): {os.path.exists(SCRIPT)}")
print(f"TRAIN_FILE ({TRAIN_FILE}): {os.path.exists(TRAIN_FILE)}")
print(f"VAL_FILE ({VAL_FILE}): {os.path.exists(VAL_FILE)}")
print(f"LOG_DIR ({LOG_DIR}): {os.path.exists(LOG_DIR)}")
print(f"DSTORE_DIR ({DSTORE_DIR}): {os.path.exists(DSTORE_DIR)}")

Checking existence for:
SCRIPT (/content/drive/MyDrive/faiss_mts_olmo/core_scripts/run_clm_chat.py): True
TRAIN_FILE (/content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_train_context_impression-finetune.jsonl): True
VAL_FILE (/content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_val_context_impression-finetune.jsonl): True
LOG_DIR (/content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/logs): True
DSTORE_DIR (/content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/datastores/allenai_OLMo-2-0425-1B-Instruct): True


The output above will tell you if each specified file or directory path currently exists on your Google Drive.

In [13]:
# ======================================
# 3. Logging
# ======================================
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = os.path.join(LOG_DIR, f"eval_knnlm_{timestamp}.log")

print(f"[INFO] Log file will be written to:\n  {log_file}\n")

[INFO] Log file will be written to:
  /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/logs/eval_knnlm_20251227_083808.log



In [14]:
# ======================================
# 4. Build the command (equivalent to bash)
# ======================================

cmd = [
    "python", "-u", SCRIPT,
    "--model_name_or_path", MODEL,
    "--train_file", TRAIN_FILE,
    "--validation_file", VAL_FILE,
    "--do_eval",
    "--eval_subset", "train",
    "--block_size", "4096",
    "--stride", "4096",
    "--per_device_eval_batch_size", "1",
    "--output_dir", OUT_DIR,
    "--dstore_dir", DSTORE_DIR,
    "--apply_chat_template",
    "--save_knnlm_dstore",
]

print("[INFO] Final command:")
print(" ".join(cmd))

[INFO] Final command:
python -u /content/drive/MyDrive/faiss_mts_olmo/core_scripts/run_clm_chat.py --model_name_or_path allenai/OLMo-2-0425-1B-Instruct --train_file /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_train_context_impression-finetune.jsonl --validation_file /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_val_context_impression-finetune.jsonl --do_eval --eval_subset train --block_size 4096 --stride 4096 --per_device_eval_batch_size 1 --output_dir /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/outputs/allenai_OLMo-2-0425-1B-Instruct --dstore_dir /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/datastores/allenai_OLMo-2-0425-1B-Instruct --apply_chat_template --save_knnlm_dstore


In [15]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"

In [16]:
# ======================================
# 5. Run the command with live logging
# ======================================

with open(log_file, "w") as lf:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1,
    )

    print("[INFO] Running...")
    for line in process.stdout:
        print(line, end="")
        lf.write(line)

process.wait()

print(f"\n[INFO] Finished. Log saved to:\n{log_file}")
print("[INFO] Datastore should now be in:")
print(DSTORE_DIR)

[INFO] Running...
2025-12-27 08:38:14.935444: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-27 08:38:14.954097: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766824694.975915   14616 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766824694.982556   14616 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766824694.999371   14616 computation_placer.cc:177] computation placer already registered. Please check linkage 